# Biomarker Selection Conventional Methods

Author: Olatomiwa Bifarin<br>
Department of Biochemistry and Molecular Biology<br>
University of Georgia<br>
Edison Lab<br>

Last edited: 21APR2021 


**Goals**: Select top 10 highest _q_-value features for metaboanalyst <br>


<a id="0"></a>

## Notebook Content

1.  [Select features with greater than 1-fold changes](#1)
2.  [t-Test Feature Selection](#2)
3.  [PLSRegression for Feature Selection](#3)
4.  [Recursive Feature Elimination for Feature Selection](#4)


In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from matplotlib import style
import scipy
#For Seaborn plots
import seaborn as sns; sns.set(style='white')
#To ignore warning
import warnings
warnings.filterwarnings('ignore')

# More sharp and legible graphics
%config InlineBackend.figure_format = 'retina'

# Set seaborn figure labels to 'talk', to be more visible. 
sns.set_context('talk', font_scale=0.8)
import statsmodels as sms
from statsmodels.stats import multitest
from statistics import mean

# Machine Learning Libraries
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import roc_auc_score


# Import Random Forest classifier, sklearn metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

# test train split and K-fold validation
from sklearn import model_selection
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

from sklearn import decomposition
from sklearn.feature_selection import RFECV

In [ ]:
print(sms.__version__)

0.10.2


In [ ]:
modelcohort = pd.read_excel('modelcohort.xlsx', index_col=0)

### Model Cohort

**a. Feature Selection** 
<a id="1"></a>

**(i)  Select features with greater than 1-fold changes**

Non-metabolic features in the `modelcohort` dataframe include the following: 
-  Sample ID
-  Patient ID
-  Collection
-  Gender
-  Race
-  BMI
-  Smoker
-  Age
-  Groups

In [ ]:
NMRMS = modelcohort.drop(['Sample ID', 'Patient ID', 'Collection', 'Gender',
                         'Race', 'BMI', 'Smoker', 'Age'], axis=1)
Control = NMRMS[(NMRMS['Groups'] == 'Control')]
RCC = NMRMS[(NMRMS['Groups'] == 'RCC')]

dfmean = pd.DataFrame({'Features':NMRMS.drop(['Groups'], axis=1).mean(axis=0).index, 
                       'Control':Control.drop(['Groups'], axis=1).mean(axis=0).values,
                       'RCC':RCC.drop(['Groups'], axis=1).mean(axis=0).values})
dfmean.shape

(7145, 3)

In [ ]:
cols = []
ctr_val = dfmean['Control']
rcc_val = dfmean['RCC']

for ctr, rcc in zip(ctr_val, rcc_val):
    ratio1 = ctr/rcc
    ratio2 = rcc/ctr
    if ratio1 > 2:
        feature = dfmean[dfmean['Control']==ctr]['Features'].values.tolist()
        cols.append(feature)
    elif ratio2 > 2:
        feature = dfmean[dfmean['Control']==ctr]['Features'].values.tolist()
        cols.append(feature)
xfold_feat = [val for sublist in cols for val in sublist] # flatten out list of list.
len(xfold_feat)

2104

**(ii) _t_-Test Feature Selection** 
<a id="2"></a>

_T-Test Function_

In [ ]:
def Ttest(metabolites, dfControl, dfTreat, alpha=0.05, var=True):
    '''
    Function conducts a T-test for the metabolites differences between two groups with 
    Benjamini-Hocberg FDR correction
    
    Inputs: 
    metabolites = A list containing names of metabolites
    dfControl = A pandas dataframe containing the control group metabolites data
    dfTreat =  A pandas dataframe containing the treatment group metabolites data
    alpha = alpha for statistical significant judgment, default 0.05
    var = If True (default), perform a standard independent 2 sample test that assumes 
    equal population variances [1]. If False, perform Welch’s t-test, which does not 
    assume equal population variance
    
    Outputs: A pandas dataframe with p-values of numerical cohort characteristics. 
    
    '''
    ttest_dict = {}
    for metabolite in metabolites:
        statistic, pvalue =  scipy.stats.ttest_ind(dfControl[metabolite], 
                                               dfTreat[metabolite], 
                                               equal_var=var)
        ttest_dict[metabolite] = pvalue 
        # a dictionary containing name of metabolites and p value after t-test
    ttest = pd.DataFrame.from_dict(ttest_dict, orient='index') # the dictionary in pandas df
    ttest_list=list(ttest_dict.values()) #values (pvalues) of ttest result in a list
    reject, pval_corrected, _, _ = sms.stats.multitest.multipletests(ttest_list, 
                                                                 alpha=alpha, 
                                                                 method='fdr_bh')
    ttest_results = pd.DataFrame({'Metabolite': metabolites, 'T-test p-value': ttest_list, 
                              'FDR p-value': pval_corrected, 'Reject H0': reject})
    Table = ttest_results.sort_values(by=['FDR p-value'])
    return Table

In [ ]:
# '''List of final metabolites set'''
metabolite_list = xfold_feat

# '''List of final metabolites set'''
ttest_result = Ttest(metabolite_list, Control, RCC, alpha=0.05, var=True)

# '''List of Statistically relevant metabolites'''
# Select metabolites with <0.05 FDR
stat_sig = ttest_result.loc[ttest_result['Reject H0'] == True]

# Select metabolites with <0.05 t-test
#stat_sig = ttest_result.loc[ttest_result['T-test p-value'] <= 0.05]
print("The total number of significant metabolites/Features is: ", len(stat_sig))

The total number of significant metabolites/Features is:  435


In [ ]:
Feature = stat_sig['Metabolite'].values.tolist(); # stat MS significant metabolites
MLfeatures = NMRMS[Feature] # p<0.05 metabolites
MLfeatures =(MLfeatures - MLfeatures.mean(axis=0))/MLfeatures.std(axis=0) #autoscaling

In [ ]:
MLfeatures.shape

(62, 435)

Top ten features with the highest q-values 

In [ ]:
stat_sig.head(10)

,Metabolite,T-test p-value,FDR p-value,Reject H0
815,2102,7.480753e-08,0.000157,True
1374,3872,2.707590e-07,0.000285,True
1321,3675,4.976800e-07,0.000349,True
1338,3757,7.746128e-07,0.000407,True
1692,5383,1.280455e-06,0.000441,True
283,720,1.466276e-06,0.000441,True
1880,6261,1.048271e-06,0.000441,True
1887,6276,2.283329e-06,0.000534,True
1503,4401,2.182275e-06,0.000534,True
1430,4080,2.719035e-06,0.000554,True


In [ ]:
conventional_MLfeatures = MLfeatures[list(stat_sig['Metabolite'][:10])]

In [ ]:
metaboanalyst=NMRMS.filter([2102, 3872, 3675, 3757, 5383, 720, 6261, 6276, 4401, 4080, 'Groups'], 
                           axis=1)

In [ ]:
#metaboanalyst.T.to_excel("metaboanalyst.xlsx")

### Overlapping Metabolomic Features Selected by Both Univariate Methods and ML Methods

In [ ]:
top_q_features = list(stat_sig['Metabolite'][:10])
top_q_features

[2102, 3872, 3675, 3757, 5383, 720, 6261, 6276, 4401, 4080]

In [ ]:
ML_markers = [720, 1481, 2102, 3141, 3675, 3804, 3872, 4080, 6261, 6262]

In [ ]:
def intersection(lst1, lst2):
    lst3 = [value for value in lst1 if value in lst2]
    return lst3

6 metabolomic features selected by both univariate and ML methods.

In [ ]:
intersection(top_q_features, ML_markers )

[2102, 3872, 3675, 720, 6261, 4080]

In [ ]:
{2102: "dibutylamine", 3872: "4.05_973.6",
 3675: "1.18_87.0641", 720: "2-phenylacetamide", 
 6261: "2.59_314.1", 4080: "0.82_406.05"}

{720: '2-phenylacetamide',
 2102: 'dibutylamine',
 3675: '1.18_87.0641',
 3872: '4.05_973.6',
 4080: '0.82_406.05',
 6261: '2.59_314.1'}

In [ ]:
def Diff(li1, li2):
    return (list(list(set(li1)-set(li2)) + list(set(li2)-set(li1))))

In [ ]:
Diff(top_q_features, intersection(top_q_features, ML_markers ) )

[4401, 6276, 3757, 5383]